In [2]:
library(NELSI)
for(f in dir('../R/', pattern = 'R$')){ 
    print(f)
    source(paste0('../R/', f))
}

[1] "get.chisq.R"
[1] "get.het.seq.R"
[1] "get.seq.R"
[1] "multlik.R"
[1] "run.mb.R"
[1] "run.phyml.bootstrap.R"
[1] "run.phyml.R"


In [3]:
boot_folders <- dir(pattern = 'Lboot.+_folder')

In [15]:
out_matrix <- matrix(NA, length(boot_folders), 5)
colnames(out_matrix) <- c('multlik', 'delta', 'chisq', 'TL', 'node_support')

In [23]:
for(i in 1:length(boot_folders)){
    b <- boot_folders[i]
    sims <- dir(b, pattern = 'phy$')
    emp_data <- read.dna(gsub('_folder', '.phy', b))
    emp_chisq <- get.chisq(emp_data)
    phyml_emp_data <- run.phyml(gsub('_folder', '.phy', b))
    pvals_mat <- matrix(NA, length(sims), 5)
    for(f in 1:length(sims)){
        phyml_temp <- run.phyml(paste0(b, '/', sims[f]))
        dat_temp <- read.dna(paste0(b, '/', sims[f]))
        chisq_temp <- get.chisq(al = dat_temp)
        pvals_mat[f, ] <- c(phyml_temp$uncLikelihood, phyml_temp$uncLikelihood - phyml_temp$likelihood, 
                            chisq_temp, phyml_temp$treeLength, phyml_temp$meanNodeSupport)
    }
    pvals <- c(sum(phyml_emp_data$uncLikelihood > pvals_mat[,1]), 
               sum((phyml_emp_data$uncLikelihood - phyml_emp_data$likelihood) > pvals_mat[,2]),
              sum(emp_chisq > pvals_mat[,3]), sum(phyml_emp_data$treeLength > pvals_mat[,4]), 
              sum(phyml_emp_data$meanNodeSupport > pvals_mat[,5]))
    out_matrix[i, ] <- pvals
}

[1] "nonstatlong.1.Lboot.gtrg.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep1.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep10.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep100.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep11.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep12.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep13.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep14.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep15.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep16.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep17.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep18.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep19.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep2.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep20.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep21.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep22.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep23.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep24.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep25.phy"
[1] "nonstatlong.1.Lboot.gtrg_folder/rep26.phy"
[1] "n

In [21]:
pvals

[1] 55 52 69 39 70

In [24]:
write.table(out_matrix, file = 'out_pvals.csv', row.names = F, sep = ',')